In [3]:
from cloudvolume import CloudVolume
dir = "precomputed://https://dlab-colm.neurodata.io/2021_04_08/gad2cre_tph2flp_con_fon_8291/642"
mip = 5

vol = CloudVolume(dir, parallel=True, progress=True, mip=mip)
image = vol[:,:,:]

In [2]:
image.shape

(116, 160, 4800, 1)

In [4]:
image.shape

(232, 320, 4800, 1)